In [ ]:
"""

Read in mseed data files, resample at 100Hz (if necessary).
Chop into 30s segments and process (demean, normalize) for use in PhaseNet.

INPUTS:
-file containing names of miniseed files

OUTPUTS:
-collection of miniseed files containing 30s snippets of processed seismic data

"""

In [ ]:
import numpy as np
import math as m
from obspy import read
from obspy import UTCDateTime
import os

In [ ]:
### CHANGE THESE!!! ###

# File containing names of stations
infile = "fname_PortlandStats.csv"

# path to directory containing mseed data
dirpath = "./PNSN_data/"

# First day of deployment period (keep the same for all stations in the experiment)
dat0 = UTCDateTime("2022-05-27T00:00:00.000")

# Flag to check for resampling (1=check files and resample, if necessary)
resamp_flag = 1

# Flag to write PhaseNet inputs (1=write miniseed inputs for PhaseNet)
phsnet_flag = 1
# path to directory where output waveforms formatted for phaseNet will be saved
pndir = "./phaseNet_input_waveforms/"

# Length of output file for PhaseNet
rec_len = 3001

# number of samples to overlap between successive PhaseNet input miniseed files (100 samples = 1 second)
# If sample length is 30s, overlap should not be >1500 (i.e., 15 s)
overlap = 1500 

#######################

In [ ]:
# Read in list of stations
stafs = np.loadtxt(infile,delimiter=',',skiprows=1,dtype=str)

# Read in mseed files, check for sampling, and resample/rewrite, if necessary.
# Then write sliced, processed input files for PhaseNet.
secs = 60*60*24 # seconds in a day
for i in range(len(stafs)):
    nmarr = stafs[i,0].split(".")
    stanm=nmarr[0]
    print("processing station %s"%format(stanm))
    directory = dirpath+stanm
    
    # Iterate through files in the station's directory and 
    # determine available dates and channels
    flg=0
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file (and not a subdirectoy)
        if os.path.isfile(f):
            # Break file name into chunks and determine its correponding date range and channel
            fn=filename.replace('__','.')
            fn=fn.split('.')
            datstrt = fn[4] # start date of miniseed file
            chan = fn[3]
            if(flg==0): # initialize arrays if this is the first file
                timarr=np.array([datstrt],dtype=str)
                chancnt=np.array([0,0,0],dtype=int)
                if((chan=='HHN')|(chan=='HNN')|(chan=='ENN')|(chan=='BHN')):
                    chancnt[0]=1
                if((chan=='HHE')|(chan=='HNE')|(chan=='ENE')|(chan=='BHE')):
                    chancnt[1]=1
                if((chan=='HHZ')|(chan=='HNZ')|(chan=='ENZ')|(chan=='BHZ')):
                    chancnt[2]=1
                flg=1
            else:
                flg2=0
                for j in range(len(timarr)): # check to see if the date has already been encountered
                    if(str(datstrt)==timarr[j]):
                        flg2=j
                        break
                if(flg2==0): # if date has not been encountered yet, create new entries in the arrays
                    timarr=np.vstack((timarr,str(datstrt)))
                    chancnt=np.vstack((chancnt,[[0,0,0]]))
                    if((chan=='HHN')|(chan=='HNN')|(chan=='ENN')|(chan=='BHN')):
                        chancnt[-1,0]=1
                    if((chan=='HHE')|(chan=='HNE')|(chan=='ENE')|(chan=='BHE')):
                        chancnt[-1,1]=1
                    if((chan=='HHZ')|(chan=='HNZ')|(chan=='ENZ')|(chan=='BHZ')):
                        chancnt[-1,2]=1
                else: # if date has been encountered, update the channel count
                    if((chan=='HHN')|(chan=='HNN')|(chan=='ENN')|(chan=='BHN')):
                        chancnt[j,0]=1
                    if((chan=='HHE')|(chan=='HNE')|(chan=='ENE')|(chan=='BHE')):
                        chancnt[j,1]=1
                    if((chan=='HHZ')|(chan=='HNZ')|(chan=='ENZ')|(chan=='BHZ')):
                        chancnt[j,2]=1
    # sum the channel count for each date to make sure that you have data for all three channels
    chansum=np.sum(chancnt,axis=1)
    ind3chan=np.argwhere(chansum==3)
    nind3chan=np.argwhere(chansum!=3)
    if(len(nind3chan)>0):
        print("Some channels missing from these dates. Data from these dates will not be used.")
        print(timarr[nind3chan])
    timarr=timarr[ind3chan]
    
    # Begin processing and saving data, doing quality checks along the way.
    nmarr = stafs[i,0].split(".")
    nmstr = nmarr[0]+"."+nmarr[1]
    if(phsnet_flag==1):
        isExist = os.path.exists(pndir+nmstr)
        if not isExist:
            os.makedirs(pndir+nmstr)
        ofil = open(pndir+nmstr+"/fname_"+nmstr+".csv","w")
        ofil.write("fname,E,N,Z\n")
    for j in range(len(timarr)):
        print('station '+stanm+', date '+str(j+1)+' of '+str(len(timarr)))
        datstrt=UTCDateTime(timarr[j][0][0])
        day_string = str(int((datstrt-dat0)/secs))

        # Read in data
        st = read(directory+'/*'+stanm+'*'+timarr[j][0][0]+'_*.mseed')
        st.merge(fill_value='latest') # merging fills gaps in data with last value
        delt = st[0].stats.delta
        if(resamp_flag==1):
            if(delt!=0.01):
                st.resample(100)
        if(phsnet_flag==1):
            # Get first time window with data for the day
            delt = st[0].stats.delta
            t0 = st[0].stats.starttime
            inc0 = m.ceil((UTCDateTime(t0)-datstrt)/((rec_len-1)*delt))
            samp = inc0*(rec_len-1)*delt
            inc = inc0
            # Process and save each chunk
            for k in range(int(secs/((rec_len-1-overlap)*delt))):
                strt = datstrt + samp
                endd = strt + ((rec_len-1)*delt)
                rec = st.slice(strt,endd)
                if(len(rec)<3): # Delete chunk if channels are missing
                    samp += (rec_len-1-overlap)*delt
                    inc += 1
                    del rec
                    continue
                # Demean traces, then normalize by their standard devations
                for ll in range(3):
                    rec[ll] = rec[ll].detrend('demean')
                    rec[ll].data = rec[ll].data/rec[ll].std()
                # Specify that the output data have float32 dtype.
                for tr in rec:
                    tr.data = np.require(tr.data, dtype=np.float32)
                # Save mseed files
                rec.write(pndir+nmstr+'/'+nmstr+'_'+day_string+'_'+str(inc),format="MSEED",encoding='FLOAT32')
                ofil.write("%s_%s_%s,%s,%s,%s\n"%(nmstr,day_string,str(inc),stafs[i,1],stafs[i,2],stafs[i,3]))
                samp += (rec_len-1-overlap)*delt
                inc += 1

    if(phsnet_flag==1):
        ofil.close()
            